In [2]:
import pandas as pd
from pygris.geocode import batch_geocode

In [ ]:
sources = pd.read_csv("./etl/conf/sources.csv")
frame = pd.read_excel(sources.iloc[-1]["source_file"])

responses = batch_geocode(frame, id_column="License Number", address="Address", 
                          city="City", state="State", zip="Zip Code")

In [ ]:
# Remember that all we want is the response columns

id_column = "License Number"
merged = frame.merge(responses.sample(len(responses) - 30), left_on=id_column, right_on="id", how="left")


merged[~merged["id"].isna()][responses.columns]

In [31]:
responses.columns

Index(['id', 'address', 'status', 'match_quality', 'matched_address',
       'tiger_line_id', 'tiger_side', 'state', 'county', 'tract', 'block',
       'longitude', 'latitude'],
      dtype='object')

In [ ]:
# The input df and the output df have the same columns sometimes

accumulation = pd.DataFrame(
    data=[], columns=[id_column, 'address', 'status', 'match_quality', 'matched_address',
    'tiger_line_id', 'tiger_side', 'state', 'county', 'tract', 'block',
    'longitude', 'latitude', 'start_date', 'end_date']
)

# there is no ambiguity per iteration on the id_column 

seen = pd.DataFrame(
    data=[], columns=[id_column, 'address', 'city', 'state', 'zip_code']
)

# Accumulation DF -- Starts empty. This is only what is returned from the geocoder (which includes id_column [rename this]) + start_date, end_date
# Seen DF -- Starts empty. This has all the columns that are provided to 'batch_geocode': [id_column, address, city, state, zip_code]

# Input DF -> 
# 1. see if the address has been geocoded using the seen df (using merge)
#    - IF in seen
#          we want to copy the rows from accumulation, adding them with new start_date, end_date
#    - ELSE
#          send to geocoder